# Iceberg - Cidades Brasileiras

Este notebook demonstra a criação, leitura e manipulação de uma tabela Iceberg usando o dataset `cidades_brasileiras.csv`.

In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("IcebergExample")
    .master("local[*]")
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.4.2")
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.local.type", "hadoop")
    .config("spark.sql.catalog.local.warehouse", "./output/iceberg-warehouse")  
    .getOrCreate()
)



25/04/24 12:03:15 WARN Utils: Your hostname, edsatc resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/04/24 12:03:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ed/.cache/pypoetry/virtualenvs/eng-dados-spark-EC3tKTXJ-py3.11/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ed/.ivy2/cache
The jars for the packages stored in: /home/ed/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-96565e0f-ce57-4d14-a900-f11453d7bc56;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.4.2 in central
:: resolution report :: resolve 188ms :: artifacts dl 2ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.4.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spar

In [2]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("cidade", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("sigla", StringType(), True),
    StructField("ibge", IntegerType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True)
])

In [3]:
df = spark.read.csv("../data/cidades_brasileiras.csv", header=True, schema=schema)
df.show(5)

+---+-------------------+------------+-----+-------+----------+----------+
| id|             cidade|      estado|sigla|   ibge|  latitude| longitude|
+---+-------------------+------------+-----+-------+----------+----------+
|  1|    ABADIA DE GOIÁS|       GOIÁS|   GO|5200050|-16.757264| -49.44122|
|  2|ABADIA DOS DOURADOS|MINAS GERAIS|   MG|3100104|-18.491063|-47.406365|
|  3|          ABADIÂNIA|       GOIÁS|   GO|5200100|-16.194723|-48.706812|
|  4|             ABAETÉ|MINAS GERAIS|   MG|3100203|-19.156683|-45.448121|
|  5|         ABAETETUBA|        PARÁ|   PA|1500107| -1.721828|-48.878843|
+---+-------------------+------------+-----+-------+----------+----------+
only showing top 5 rows



In [4]:
df.writeTo("local.cidades_iceberg").using("iceberg").createOrReplace()


In [5]:
spark.sql('''SELECT cidade, sigla FROM local.cidades_iceberg WHERE sigla = 'SC';''').show(10)


+----------------+-----+
|          cidade|sigla|
+----------------+-----+
|   ABDON BATISTA|   SC|
|    ABELARDO LUZ|   SC|
|      AGROLÂNDIA|   SC|
|      AGRONÔMICA|   SC|
|       ÁGUA DOCE|   SC|
|ÁGUAS DE CHAPECÓ|   SC|
|     ÁGUAS FRIAS|   SC|
|    ÁGUAS MORNAS|   SC|
|  ALFREDO WAGNER|   SC|
| ALTO BELA VISTA|   SC|
+----------------+-----+
only showing top 10 rows



In [13]:
spark.sql("""
    INSERT INTO local.cidades_iceberg VALUES
    (9999, 'Cidade Exemplo', 'Estado Exemplo', 'EX', 9999999, -10.1234, -50.5678)
""")

spark.sql("SELECT * FROM local.cidades_iceberg WHERE id = 9999").show()


+----+--------------+--------------+-----+-------+--------+---------+
|  id|        cidade|        estado|sigla|   ibge|latitude|longitude|
+----+--------------+--------------+-----+-------+--------+---------+
|9999|Cidade Exemplo|Estado Exemplo|   EX|9999999|-10.1234| -50.5678|
+----+--------------+--------------+-----+-------+--------+---------+



In [14]:
spark.sql("UPDATE local.cidades_iceberg SET cidade = 'Cidade Atualizada' WHERE id = 9999")
spark.sql("SELECT * FROM local.cidades_iceberg WHERE id = 9999").show()

+----+-----------------+--------------+-----+-------+--------+---------+
|  id|           cidade|        estado|sigla|   ibge|latitude|longitude|
+----+-----------------+--------------+-----+-------+--------+---------+
|9999|Cidade Atualizada|Estado Exemplo|   EX|9999999|-10.1234| -50.5678|
+----+-----------------+--------------+-----+-------+--------+---------+



In [15]:
spark.sql("DELETE FROM local.cidades_iceberg WHERE id = 9999")
spark.sql("SELECT * FROM local.cidades_iceberg WHERE id = 9999").show()

+---+------+------+-----+----+--------+---------+
| id|cidade|estado|sigla|ibge|latitude|longitude|
+---+------+------+-----+----+--------+---------+
+---+------+------+-----+----+--------+---------+



In [16]:
df_final = spark.sql("SELECT * FROM local.cidades_iceberg")
df_final.show(5)

+---+-------------------+------------+-----+-------+----------+----------+
| id|             cidade|      estado|sigla|   ibge|  latitude| longitude|
+---+-------------------+------------+-----+-------+----------+----------+
|  1|    ABADIA DE GOIÁS|       GOIÁS|   GO|5200050|-16.757264| -49.44122|
|  2|ABADIA DOS DOURADOS|MINAS GERAIS|   MG|3100104|-18.491063|-47.406365|
|  3|          ABADIÂNIA|       GOIÁS|   GO|5200100|-16.194723|-48.706812|
|  4|             ABAETÉ|MINAS GERAIS|   MG|3100203|-19.156683|-45.448121|
|  5|         ABAETETUBA|        PARÁ|   PA|1500107| -1.721828|-48.878843|
+---+-------------------+------------+-----+-------+----------+----------+
only showing top 5 rows

